# Data Loading

In [181]:
import os

def get_file_names(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)
    # Filter out directories, leaving only files
    file_names = [file for file in files if os.path.isfile(os.path.join(folder_path, file))]
    return file_names

folder_path = './marked_data'
file_names = get_file_names(folder_path)
print("File names in the folder:")
for file_name in file_names:
    print(file_name)

File names in the folder:
result_with_parameters_0.csv
result_with_parameters_1.csv
result_with_parameters_2.csv


In [182]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = list()
for i, file_name in enumerate(file_names):
    temp = pd.read_csv(f'./marked_data/{file_name}')
    temp = temp.drop(["index", "past_OBV", "upper_ST", "lower_ST"], axis=1)
    data_for_analysis = temp
    y, X = temp["result"], temp.drop(["result"], axis=1)
    data.append([y, X])
    
data[2][1]

,close,open,high,low,volume,past_EMA,current_EMA,current_RSI,past_RSI,past_MACD,current_MACD,current_OBV,upper_BB,lower_BB,%K_StochRSI,ATR,up_Aroon,low_Aroon
0,270.22,270.22,270.61,270.20,70,-0.063712,0.436842,13.189032,49.574189,0.023510,0.001368,9.200000,270.751797,269.990203,11.392405,1.383741,92.0,28.0
1,270.60,270.60,270.60,270.23,36,0.436842,0.852394,17.223395,13.189032,0.001368,0.011350,8.400000,270.777576,269.989424,35.443038,1.382825,88.0,24.0
2,270.60,270.60,270.61,270.07,236,0.852394,1.223820,17.223395,17.223395,0.011350,0.016666,12.533333,270.780016,270.043984,35.443038,1.393303,84.0,20.0
3,270.60,270.25,270.60,270.07,52,1.223820,1.555370,17.223395,17.223395,0.016666,0.018742,7.733333,270.797415,270.042585,35.443038,1.402416,80.0,16.0
4,270.11,270.60,270.60,270.09,136,1.555370,1.892675,16.221694,17.223395,0.018742,-0.012636,3.466667,270.805300,270.001700,0.000000,1.409589,76.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21564,276.07,276.06,276.10,276.06,930,-0.077712,-0.074969,51.333664,51.094821,-0.010906,-0.009423,48.133333,276.205463,276.010537,1.754386,1.031477,48.0,12.0
21565,276.06,276.07,276.19,276.06,2371,-0.074969,-0.071588,51.081684,51.333664,-0.009423,-0.008656,-168.466667,276.206069,276.007931,0.877193,1.038045,44.0,8.0
21566,276.10,276.13,276.14,276.06,204,-0.071588,-0.071903,52.032965,51.081684,-0.008656,-0.005167,-183.466667,276.206108,276.010892,4.424779,1.040842,40.0,4.0
21567,276.10,276.09,276.10,276.09,5,-0.071903,-0.072117,52.032965,52.032965,-0.005167,-0.002696,-128.266667,276.206026,276.013974,4.424779,1.038786,36.0,8.0


In [183]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# get data
divided_data = list()
for data_train in data:
    divided_data.append(train_test_split(data_train[1], data_train[0], test_size=0.20, random_state = 2020, stratify=data_train[0]))

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 2020, stratify=y)

# Scale Data

In [184]:
import numpy as np
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
for i, data_train in enumerate(divided_data):
    divided_data[i][0] = ss.fit_transform(data_train[0])
    divided_data[i][1] = ss.fit_transform(data_train[1])
    
X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []

for i, data_train in enumerate(divided_data):
    X_train_list.append(data_train[0])
    X_test_list.append(data_train[1])
    y_train_list.append(data_train[2])
    y_test_list.append(data_train[3])

X_train_scaled = np.concatenate(X_train_list, axis=0)
X_test_scaled = np.concatenate(X_test_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)
y_test = np.concatenate(y_test_list, axis=0)

X_train_scaled
# X_train_scaled = ss.fit_transform(X_train)
# X_test_scaled = ss.transform(X_test)
# y_train = np.array(y_train)

array([[ 0.71856253,  0.71737162,  0.71565183, ..., -0.66004104,
         0.96219617, -1.34887654],
       [ 0.55370204,  0.55343777,  0.54955531, ..., -0.76041134,
         0.49027163, -1.34887654],
       [ 0.33976458,  0.33303253,  0.33439569, ..., -0.12614371,
        -1.04348313,  0.90823864],
       ...,
       [-1.77603152, -1.7693378 , -1.81371463, ..., -0.74206906,
         0.71508586, -1.13595917],
       [ 0.96269679,  0.94874084,  0.98251408, ..., -0.24320639,
        -0.48701134,  0.79984405],
       [ 0.40265759,  0.4365401 ,  0.41914613, ..., -0.39699525,
         0.23424698, -0.53102067]])

# Over Sampling

In [185]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

adasyn = ADASYN()
#smote = SMOTE()

# Fit SMOTE to the training data
X_train_scaled, y_train = adasyn.fit_resample(X_train_scaled, y_train)
X_train_scaled

array([[ 0.71856253,  0.71737162,  0.71565183, ..., -0.66004104,
         0.96219617, -1.34887654],
       [ 0.55370204,  0.55343777,  0.54955531, ..., -0.76041134,
         0.49027163, -1.34887654],
       [ 0.33976458,  0.33303253,  0.33439569, ..., -0.12614371,
        -1.04348313,  0.90823864],
       ...,
       [-1.54643631, -1.5492015 , -1.39462384, ...,  1.39998775,
        -1.01389715,  0.48322588],
       [-1.56228098, -1.51781816, -1.41009931, ...,  1.36838693,
        -0.85954108,  0.54589105],
       [-1.58208984, -1.48135002, -1.42962127, ...,  1.29761393,
        -1.09031956,  0.19035705]])

# Undersempling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Instantiate RandomUnderSampler
undersampler = RandomUnderSampler()

# Fit undersampler to the training data
X_train_scaled, y_train = undersampler.fit_resample(X_train_scaled, y_train)
X_train_scaled

# Learning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

rfc = RandomForestClassifier()
rfc.fit(X_train_scaled, y_train)
display(rfc.score(X_train_scaled, y_train))
display(rfc.score(X_test_scaled, y_test))

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

feats = {}
for feature, importance in zip(data_for_analysis.drop("result", axis=1).columns, rfc.feature_importances_):
    feats[feature] = importance
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-Importance'})
importances = importances.sort_values(by='Gini-Importance', ascending=False)
importances = importances.reset_index()
importances = importances.rename(columns={'index': 'Features'})
sns.set(font_scale = 5)
sns.set(style="whitegrid", color_codes=True, font_scale = 1.7)
fig, ax = plt.subplots()
fig.set_size_inches(30,15)
sns.barplot(x=importances['Gini-Importance'], y=importances['Features'], data=importances, color='skyblue')
plt.xlabel('Importance', fontsize=25, weight = 'bold')
plt.ylabel('Features', fontsize=25, weight = 'bold')
plt.title('Feature Importance', fontsize=25, weight = 'bold')
display(plt.show())
display(importances)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

pca_test = PCA(n_components=12)
pca_test.fit(X_train_scaled)
sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=10, ymin=0, ymax=1)
display(plt.show())
evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.head(10))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(rfc, 
                        param_dist, 
                        n_iter = 100,
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(X_train_scaled, y_train)
rs.best_params_

In [ ]:
"""
just in case
{'n_estimators': 800,
 'min_samples_split': 28,
 'min_samples_leaf': 12,
 'max_features': 'sqrt',
 'max_depth': 14,
 'bootstrap': True}
"""


from sklearn.metrics import confusion_matrix, precision_score

# y_pred_probs = rfc.predict_proba(X_test_scaled)[:, 1]
# 
# new_threshold = 0.8  # Example: setting a higher threshold
# 
# # Adjust the predicted labels based on the new threshold
# y_pred = (y_pred_probs >= new_threshold).astype(int)
y_pred = rfc.predict(X_test_scaled)

conf_matrix_baseline = pd.DataFrame(confusion_matrix(y_test, y_pred), index = ['actual 0', 'actual 1'], columns = ['predicted 0', 'predicted 1'])

display(conf_matrix_baseline)
display('Baseline Random Forest recall score', recall_score(y_test, y_pred))
display('Baseline Random Forest precision score', precision_score(y_test, y_pred))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score

# Create a Gradient Boosting Classifier instance
gbm = GradientBoostingClassifier()

# Fit the model to the training data
gbm.fit(X_train_scaled, y_train)

# Evaluate the model on the training and test data
train_score = gbm.score(X_train_scaled, y_train)
test_score = gbm.score(X_test_scaled, y_test)

print("Training Accuracy:", train_score)
print("Test Accuracy:", test_score)

In [186]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Instantiate the GBM classifier
gbm = GradientBoostingClassifier()

scorer = make_scorer(precision_score)

# Instantiate GridSearchCV with precision as the scoring metric
grid_search = GridSearchCV(gbm, param_grid, cv=5, scoring=scorer, n_jobs=-1)

# Perform Grid Search
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Evaluate on test set
test_score = grid_search.score(X_train_scaled, y_test)
print("Test Accuracy:", test_score)

Best Parameters: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200}


ValueError: Found input variables with inconsistent numbers of samples: [12243, 88996]

In [192]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Evaluate on test set
test_score = grid_search.score(X_test_scaled, y_test)
print("Test Accuracy:", test_score)

y_pred_test = grid_search.predict(X_test_scaled)

display(conf_matrix_baseline)
print("Test Recall Score:", recall_score(y_test, y_pred_test))
display('Test precision score', precision_score(y_test, y_pred_test))

Best Parameters: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200}
Test Accuracy: 0.15275634731378876


,predicted 0,predicted 1
actual 0,7439,3671
actual 1,161,972


Test Recall Score: 0.9611650485436893


'Test precision score'

0.15275634731378876

In [ ]:
# Calculate recall score on the test data
y_pred_test = gbm.predict(X_test_scaled)
recall = recall_score(y_test, y_pred_test)

conf_matrix_baseline = pd.DataFrame(confusion_matrix(y_test, y_pred_test), index = ['actual 0', 'actual 1'], columns = ['predicted 0', 'predicted 1'])

display(conf_matrix_baseline)
print("Test Recall Score:", recall)
display('Test precision score', precision_score(y_test, y_pred))